In [ ]:

import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import nltk
from nltk.stem.lancaster import LancasterStemmer
import random
import time
import os

# --- 1. Data Setup (Loading intents.json from file) ---
INTENTS_FILE = '/content/intent.json'

try:
    # Check if the file exists in the current directory
    if not os.path.exists(INTENTS_FILE):
        print(f"Error: The file '{INTENTS_FILE}' was not found.")
        print("Please upload your 'Intent.json' file to the Colab session's file panel.")
        raise FileNotFoundError(f"{INTENTS_FILE} not found.")

    with open(INTENTS_FILE, encoding='utf-8') as file:
        data = json.load(file)
    print(f"Successfully loaded data from {INTENTS_FILE}.")

except Exception as e:
    print(f"An error occurred while loading the data: {e}")
    exit()


# Initialize stemmer and download NLTK punkt
stemmer = LancasterStemmer()
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading NLTK punkt...")
    nltk.download('punkt')

# Try to find punkt_tab and download if not found
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading NLTK punkt_tab...")
    nltk.download('punkt_tab')


# --- 2. Preprocessing and Feature Extraction ---

words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',', "'s", "'re", "'m"]

for intent_data in data['intents']:
    tag = intent_data['tag']
    classes.append(tag)

    for pattern in intent_data['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, tag))

words = [stemmer.stem(w.lower()) for w in words if w.lower() not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(f"Total unique stemmed words (Vocabulary size): {len(words)}")
print(f"Total intents/classes: {len(classes)}")
print(f"Number of documents/patterns: {len(documents)}")

# Create training data
training = []
output = []
output_empty = [0] * len(classes)

# Training set, bag of words for each pattern
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append(bag)
    output.append(output_row)

# Convert training data to numpy arrays
train_x = np.array(training)
train_y = np.array(output)


# --- 3. Model Definition and Training ---

# Build the Neural Network Model
# Using a 3-layer fully connected network (Deep Neural Network)
model = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print("\nStarting model training (200 epochs)...")
# Train the model (set verbose=1 to see training progress)
history = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=0)
print("Model training complete!")
print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")


# --- 4. Prediction Functions ---

def clean_up_sentence(sentence):
    """Tokenizes and stems the sentence."""
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words, show_details=False):
    """Creates a bag of words (vector) for the input sentence."""
    sentence_words = clean_up_sentence(sentence)
    # Bag of words matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def classify_intent(sentence):
    """Predicts the intent tag for the sentence."""
    # Generate predictions from the model
    input_data = bag_of_words(sentence, words)
    try:
        results = model.predict(np.array([input_data]), verbose=0)[0]
    except Exception:
        return []

    # Filter out predictions below a threshold
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    # Sort by probability in descending order
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # Return tuple of intent and probability
    return return_list

def get_response(intents_list, intents_json):
    """Gets a random response based on the highest-confidence intent."""
    if not intents_list:
        return "I'm sorry, I don't understand that. Can you rephrase?"

    tag = intents_list[0][0]
    list_of_intents = intents_json['intents']

    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result
    return "Something went wrong fetching the response."


# --- 5. Chat Loop Interface ---

print("\n" + "="*70)
print("🌟 WELCOME TO DHAKA HERITAGE CHATBOT 🌟")
print("="*70)
print("\n🏛️  Dhaka's Historical Sites - Places You Can Learn About:")
print("• Lalbagh Fort – Mughal-era fort and palace")
print("• Ahsan Manzil – Pink palace, former residence of the Nawab of Dhaka")
print("• Curzon Hall – Historic building of Dhaka University")
print("• Dhakeshwari Temple – Ancient Hindu temple, national temple of Bangladesh")
print("• Armenian Church of Dhaka – Oldest Christian church in Dhaka")
print("• Old Dhaka City Walls – Remains of historic city fortifications")
print("• Shahid Minar – National monument for Language Movement martyrs")
print("\n" + "="*70)
print(f"🤖 Model trained on {len(classes)} intents. Start chatting!")
print("💡 Type 'quit' to exit the chat.")
print("="*70 + "\n")

while True:
    try:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            break

        # 1. Classify the user input (get intent tag)
        intent_results = classify_intent(user_input)

        # 2. Get a random response based on the intent
        bot_response = get_response(intent_results, data)

        print(f"Bot: {bot_response}")

    except EOFError:
        # Handles case where input is closed (common in scripted environments)
        print("\nChatBot session ended (EOF).")
        break
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        time.sleep(1)
        break

print("\n" + "="*70)
print("Thank you for using Dhaka Heritage Chatbot! 👋")
print("Hope you learned something interesting about Dhaka's rich history!")
print("="*70)

Successfully loaded data from /content/intent.json.
Total unique stemmed words (Vocabulary size): 506
Total intents/classes: 209
Number of documents/patterns: 1361

Starting model training (200 epochs)...
Model training complete!
Final training accuracy: 0.8376

🌟 WELCOME TO DHAKA HERITAGE CHATBOT 🌟

🏛️  Dhaka's Historical Sites - Places You Can Learn About:
• Lalbagh Fort – Mughal-era fort and palace
• Ahsan Manzil – Pink palace, former residence of the Nawab of Dhaka
• Curzon Hall – Historic building of Dhaka University
• Dhakeshwari Temple – Ancient Hindu temple, national temple of Bangladesh
• Armenian Church of Dhaka – Oldest Christian church in Dhaka
• Old Dhaka City Walls – Remains of historic city fortifications
• Shahid Minar – National monument for Language Movement martyrs

🤖 Model trained on 209 intents. Start chatting!
💡 Type 'quit' to exit the chat.

Bot: Curzon Hall is primarily an academic building, but visitors can usually access the exterior and certain public areas. 